You are an expert Python developer specializing in the Databricks environment. Your task is to create a complete Python script to be executed within a Databricks notebook. The script must perform the following operations:
1.	Data Retrieval from SpaceX API:
o	Interact with the SpaceX v3 REST API (https://api.spacexdata.com/v3).
o	Retrieve data from two specific endpoints: 
	All launches: https://api.spacexdata.com/v3/launches
	All rockets: https://api.spacexdata.com/v3/rockets
o	Handle potential errors during the API calls (e.g., timeouts, non-200 status codes).
2.	Merge Operation:
o	Perform a "merge" (or join) operation between the launches data and the rockets data.
o	Merge Logic: For each launch record, add the rocket's name (rocket_name) from the rockets dataset. The match should be based on the rocket.rocket_id field present in each launch record corresponding to the rocket_id field in each rocket record.
o	The final result should be a list of dictionaries, where each dictionary represents a launch enriched with its corresponding rocket name.
3.	Control Parameters and Debugging:
o	Include variables at the beginning of the script to define the API endpoint URLs, making them easily modifiable.
o	Use Python's standard logging module to provide informative output during execution. Configure logging to display messages at the INFO level.
o	Log key messages such as: starting data retrieval, number of launches retrieved, number of rockets retrieved, starting merge operation, number of records after merge, starting upload to httpbin, upload outcome.
4.	Execution Time Measurement:
o	Code Execution Time: Measure the time taken to perform the main operations (data retrieval + merge). Print this time after the merge operation is complete.
o	Pipeline Execution Time: Measure the total execution time of the entire script (from the beginning until after the upload to httpbin). Print this total time at the end of the script. Use Python's time module.
5.	Upload Result:
o	Take the resulting list from the merge operation.
o	Serialize it into JSON format.
o	Make an HTTP POST request to the https://httpbin.org/post endpoint, sending the resulting JSON data in the request body.
o	Verify the response from httpbin.org (e.g., check the status code) and log the outcome of the upload operation.

In [0]:
# Databricks Notebook: SpaceX Data Pipeline
# Author: ChatGPT
# Purpose: Retrieve, merge, and upload SpaceX launch and rocket data.

# ---------------------
# Imports
# ---------------------
import requests
import logging
import json
import time

# ---------------------
# Configuration
# ---------------------
# Control Parameters
LAUNCHES_URL = "https://api.spacexdata.com/v3/launches"
ROCKETS_URL = "https://api.spacexdata.com/v3/rockets"
UPLOAD_URL = "https://httpbin.org/post"

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# ---------------------
# Helper Functions
# ---------------------

def fetch_data(url):
    """Fetch data from a given URL with error handling."""
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        logging.error(f"Error fetching data from {url}: {e}")
        return None

def merge_launches_with_rockets(launches, rockets):
    """Merge launches with rockets based on rocket_id."""
    rocket_id_to_name = {rocket['rocket_id']: rocket['rocket_name'] for rocket in rockets}
    
    merged_data = []
    for launch in launches:
        rocket_info = launch.get('rocket', {})
        rocket_id = rocket_info.get('rocket_id')
        rocket_name = rocket_id_to_name.get(rocket_id, "Unknown Rocket")
        
        # Enrich the launch data
        enriched_launch = launch.copy()
        enriched_launch['rocket_name'] = rocket_name
        merged_data.append(enriched_launch)
    
    return merged_data

def upload_data(url, data):
    """Upload data as JSON to a given URL."""
    try:
        headers = {"Content-Type": "application/json"}
        response = requests.post(url, data=json.dumps(data), headers=headers, timeout=10)
        response.raise_for_status()
        return response
    except requests.exceptions.RequestException as e:
        logging.error(f"Error uploading data to {url}: {e}")
        return None

# ---------------------
# Main Execution
# ---------------------

# Start total execution timer
pipeline_start_time = time.time()

logging.info("Starting SpaceX Data Pipeline...")

# Data retrieval timer
data_start_time = time.time()

# Fetch launches data
logging.info("Retrieving launches data...")
launches_data = fetch_data(LAUNCHES_URL)
if launches_data is None:
    raise SystemExit("Failed to retrieve launches data. Exiting pipeline.")

logging.info(f"Retrieved {len(launches_data)} launch records.")

# Fetch rockets data
logging.info("Retrieving rockets data...")
rockets_data = fetch_data(ROCKETS_URL)
if rockets_data is None:
    raise SystemExit("Failed to retrieve rockets data. Exiting pipeline.")

logging.info(f"Retrieved {len(rockets_data)} rocket records.")

# Perform merge
logging.info("Merging launches with rocket names...")
merged_data = merge_launches_with_rockets(launches_data, rockets_data)

logging.info(f"Merge complete. Total merged records: {len(merged_data)}")

# Measure data retrieval + merge time
data_end_time = time.time()
data_elapsed_time = data_end_time - data_start_time
logging.info(f"Data retrieval and merge operations completed in {data_elapsed_time:.2f} seconds.")

# Upload merged data
logging.info("Uploading merged data to httpbin.org...")
upload_response = upload_data(UPLOAD_URL, merged_data)

if upload_response and upload_response.status_code == 200:
    logging.info("Upload successful! Response received from httpbin.org.")
else:
    logging.error("Upload failed.")

# Measure total pipeline time
pipeline_end_time = time.time()
pipeline_elapsed_time = pipeline_end_time - pipeline_start_time
logging.info(f"Total pipeline execution time: {pipeline_elapsed_time:.2f} seconds.")

print(f"Data retrieval + merge time: {data_elapsed_time:.2f} seconds")
print(f"Total pipeline execution time: {pipeline_elapsed_time:.2f} seconds")
